<a href="https://colab.research.google.com/github/galacticor/learndatascience/blob/master/kaggle/Latihan1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
melb_data = pd.read_csv("https://raw.githubusercontent.com/galacticor/Kuliah/master/DS/dataset/melb_data.csv")

In [ ]:
y = melb_data.Price
melb_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
melb_data = melb_data.dropna()
X = melb_data[melb_features]
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

def get_mae(train_X, val_X, train_y, val_y, max_leaf_nodes = None, random_state = 0):
  model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state = random_state)
  model.fit(train_X, train_y)
  val_predict = model.predict(val_X)
  mae = mean_absolute_error(val_predict, val_y)
  return mae
  
val = get_mae(X,X,y,y)
print(val)


# Bisa dilihat hasil MAE awal kalo train dari valuenya

1115.7467183128902


In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state = 1)
val = get_mae(train_X, val_X, train_y, val_y, random_state = 1)
print(val)

# Bisa dilihat hasil MAE awal kalo train dari value hasil split dipilih yang mana yang buat train dan yang mana yang buat test,
# nilai tambah gede karena ada model yang featurenya ga cocok karena kadang klo dimasukin data baru bukan dari train maka bisa error gede banget

251688.7630729503


In [ ]:
# max_leaf_nodes di sini itu jadi kedalaman maximal dari DecisionTreeRegressionnya.
# ada point dimana dia jadi titik minimum, kita bisa lakukan ternary_search atau looping buat cari 
# nilai max_leaf_nodes agar dapet mae terkecil

best_tree_size = 5
min = 9999999999999
# Write loop to find the ideal tree size from candidate_max_leaf_nodes

for x in range(100,5000, 5):
    tmp = get_mae(train_X, val_X, train_y, val_y, x)
    if tmp < min:
        min = tmp
        best_tree_size = x
    

# Store the best value of max_leaf_nodes (it will be either 5, 25, 50, 100, 250 or 500)
print(best_tree_size)

# fungsi loop bisa dioptimisasi pake TERNARY_SEARCH

250


In [ ]:
def binary_search(train_X, val_X, train_y, val_y):
  lo = 5
  hi = 1000
  while(lo <= hi):
    mid = (lo+hi)//2
    mae_mid = get_mae(train_X, val_X, train_y, val_y, mid)
    mae_mid_left = get_mae(train_X, val_X, train_y, val_y, mid-1)
    mae_mid_right = get_mae(train_X, val_X, train_y, val_y, mid+1)
    if mae_mid < mae_mid_left and mae_mid < mae_mid_right:
      return mid
    elif mae_mid > mae_mid_left and mae_mid < mae_mid_right:
      hi = mid
    else:
      lo = mid

best_tree_size = binary_search(train_X, val_X, train_y, val_y)
print(best_tree_size)


502


In [ ]:
print(get_mae(train_X, val_X, train_y, val_y,250))
print(get_mae(train_X, val_X, train_y, val_y,251))
print(get_mae(train_X, val_X, train_y, val_y,252))
print(get_mae(train_X, val_X, train_y, val_y,501))
print(get_mae(train_X, val_X, train_y, val_y,502))
print(get_mae(train_X, val_X, train_y, val_y,503))

#oke dari sini ternyata gabisa sembarangan pake binary search karena ada 2 peak berbeda

240719.92550287693
240584.78771632264
240662.78734742102
244227.00523552002
244102.19352904276
244319.88236054697


In [ ]:
# Model baru yaitu Random Forest Regression, buat menyelesaikan masalah overfitting dan underfitting
# yang ada pada DecisionTreeRegressor untuk mendapat max_leaf_nodes optimum. Caranya sama persis kaya DecisionTree
# The random forest uses many trees, and it makes a prediction by averaging the predictions of each component tree. 
# It generally has much better predictive accuracy than a single decision tree and it works well with default parameters. 
# If you keep modeling, you can learn more models with even better performance, but many of those are sensitive to getting the right parameters.

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

# Terlihat improvement MAE dari yang dipake DecisionTreeRegressor dengan RandomForestRegressor, 

190414.59149025998
